In [ ]:
import backtrader as bt
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import logging
import requests

In [ ]:

def fetch_polygon_data(ticker, timespan='day', from_date='2020-01-01', to_date='2023-12-31', api_key='YOUR_POLYGON_API_KEY'):
    """
    Fetches historical data from Polygon.io.

    Parameters:
        ticker (str): Stock ticker symbol (e.g., 'GLD', 'SLV').
        timespan (str): Timespan of the data (e.g., 'minute', 'hour', 'day').
        from_date (str): Start date in 'YYYY-MM-DD' format.
        to_date (str): End date in 'YYYY-MM-DD' format.
        api_key (str): Your Polygon.io API key.

    Returns:
        pd.DataFrame: DataFrame containing historical data.
    """
    url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{from_date}/{to_date}'
    params = {
        'adjusted': 'true',
        'sort': 'asc',
        'limit': 50000,  # Maximum number of results per request
        'apiKey': api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if 'results' not in data:
            logging.error(
                f"No results found for ticker {ticker}. Response: {data}")
            return None

        df = pd.DataFrame(data['results'])
        # Convert timestamp to datetime
        df['t'] = pd.to_datetime(df['t'], unit='ms')
        df.set_index('t', inplace=True)
        df.rename(columns={
            'o': 'Open',
            'h': 'High',
            'l': 'Low',
            'c': 'Close',
            'v': 'Volume',
            'n': 'Transactions'
        }, inplace=True)
        return df[['Open', 'High', 'Low', 'Close', 'Volume']]

    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred for ticker {ticker}: {http_err}")
    except Exception as err:
        logging.error(f"An error occurred for ticker {ticker}: {err}")

    return None


# Example usage:
# Replace 'YOUR_POLYGON_API_KEY' with your actual API key
gld_data = fetch_polygon_data(
    'GLD', from_date='2020-01-01', to_date='2023-12-31', api_key='YOUR_POLYGON_API_KEY')
slv_data = fetch_polygon_data(
    'SLV', from_date='2020-01-01', to_date='2023-12-31', api_key='YOUR_POLYGON_API_KEY')

# Check if data fetching was successful
if gld_data is None or slv_data is None:
    logging.error("Data fetching failed. Exiting the script.")
    exit()

In [ ]:

class PandasData(bt.feeds.PandasData):
    """
    Customized Pandas Data Feed for Backtrader.
    """
    params = (
        ('datetime', None),  # Use the DataFrame index as datetime
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', 'Volume'),
        ('openinterest', -1),  # No open interest data
    )


# Create data feeds for GLD and SLV
data_gld = PandasData(dataname=gld_data)
data_slv = PandasData(dataname=slv_data)

In [ ]:

class SARIMAXStrategy(bt.Strategy):
    params = (
        ('forecast_period', 1),          # Number of days to forecast ahead
        ('model_order', (1, 1, 1)),      # SARIMAX (p,d,q) order
        ('seasonal_order', (1, 1, 1, 7)),  # SARIMAX (P,D,Q,s) seasonal order
        ('threshold', 0.5),              # Threshold for making trades
        ('verbose', False),              # Enable detailed logging
    )

    def __init__(self):
        # Reference to the close prices
        self.dataclose = self.datas[0].close

        # Initialize variables to track orders
        self.order = None

        # Initialize and fit the SARIMAX model
        self.model = None
        self.model_fit = None
        self.initialize_model()

    def initialize_model(self):
        """
        Initialize and fit the SARIMAX model using historical data.
        """
        try:
            # Extract historical close prices as a pandas Series
            historical_data = pd.Series(
                [d[0] for d in self.datas[0].get(size=len(self))])

            # Fit SARIMAX model
            self.model = SARIMAX(
                historical_data,
                order=self.params.model_order,
                seasonal_order=self.params.seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            self.model_fit = self.model.fit(disp=False)

            if self.params.verbose:
                print(self.model_fit.summary())
        except Exception as e:
            logging.error(f"Error initializing SARIMAX model: {e}")

    def next(self):
        """
        Called for each new data point. Make predictions and execute trades.
        """
        if self.order:
            # Pending order execution
            return

        # Define the lookback period based on model requirements
        lookback = max(
            self.params.model_order[0], self.params.seasonal_order[3])

        if len(self) < lookback:
            # Not enough data to make a prediction
            return

        try:
            # Extract the latest historical data required for prediction
            historical_data = pd.Series(
                [d[0] for d in self.datas[0].get(size=lookback)])

            # Update and refit the model with new data (optional for better accuracy)
            self.model = SARIMAX(
                historical_data,
                order=self.params.model_order,
                seasonal_order=self.params.seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            self.model_fit = self.model.fit(disp=False)

            # Make a forecast
            forecast = self.model_fit.forecast(
                steps=self.params.forecast_period)
            predicted_price = forecast[-1]

            # Current price
            current_price = self.dataclose[0]

            if self.params.verbose:
                print(
                    f"Predicted Price: {predicted_price}, Current Price: {current_price}")

            # Trading logic based on prediction
            # Example Strategy:
            # - Buy if predicted price is higher than current price by a threshold
            # - Sell if predicted price is lower than current price by a threshold

            if predicted_price > current_price + self.params.threshold:
                # Buy signal
                if not self.position:
                    self.order = self.buy()
                    if self.params.verbose:
                        print(f"BUY EXECUTED at {current_price}")
            elif predicted_price < current_price - self.params.threshold:
                # Sell signal
                if self.position:
                    self.order = self.sell()
                    if self.params.verbose:
                        print(f"SELL EXECUTED at {current_price}")

        except Exception as e:
            logging.error(f"Error during forecasting and trading: {e}")

    def notify_order(self, order):
        """
        Notification for order status changes.
        """
        if order.status in [order.Submitted, order.Accepted]:
            # Order submitted/accepted, nothing to do
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                if self.params.verbose:
                    print(f"BUY ORDER COMPLETED at {self.buyprice}")
            elif order.issell():
                if self.params.verbose:
                    print(f"SELL ORDER COMPLETED at {order.executed.price}")

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            logging.warning(f"Order {order.Status[order.status]}")

        # Reset orders
        self.order = None

    def notify_trade(self, trade):
        """
        Notification for trade status changes.
        """
        if not trade.isclosed:
            return

        if self.params.verbose:
            print(f"OPERATION PROFIT, GROSS {trade.pnl}, NET {trade.pnlcomm}")

In [ ]:

# Initialize Cerebro engine
cerebro = bt.Cerebro()

# Add data feeds to Cerebro
cerebro.adddata(data_gld, name='GLD')
cerebro.adddata(data_slv, name='SLV')

# Add the SARIMAX strategy to Cerebro
cerebro.addstrategy(
    SARIMAXStrategy,
    forecast_period=1,
    model_order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 7),
    threshold=0.5,
    verbose=True  # Set to True for detailed logging
)

# Set initial capital
cerebro.broker.setcash(100000.0)

# Set commission - 0.1% per trade
cerebro.broker.setcommission(commission=0.001)

# Print starting portfolio value
print(f"Starting Portfolio Value: {cerebro.broker.getvalue():.2f}")

# Run the backtest
results = cerebro.run()

# Print final portfolio value
print(f"Final Portfolio Value: {cerebro.broker.getvalue():.2f}")

# Plot the results
cerebro.plot(style='candlestick')